# Ejemplo de DNNs con Boston Breast Cancer Dataset

## Carga y procesado de los datos

Primero, cargamos el dataset de un CSV disponible en Internet:

In [ ]:
!curl -O https://raw.githubusercontent.com/autonomio/datasets/master/autonomio-datasets/breast_cancer.csv

Comprobamos que nuestro fichero CSV está en el sistema de ficheros local de la máquina virtual en la que se está ejecutando Google Colab:

In [ ]:
!ls

Ahora, importamos diferentes librerías necesarias para poder procesar los datos, e importamos el CSV en un dataframe de Pandas:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('breast_cancer.csv')

Echamos un vistazo al dataset, que ya conocemos bien de las clases de Supervised Learning con Python y Scikit-Learn:

In [ ]:
dataset.head(5)

Ahora separamos las variables dependientes e independientes, nuestras muestras de las etiquetas de clasificación del dataset. Nótese qu extamos excluyendo la última columna de las muestras:

In [ ]:
X = dataset.iloc[:, 2:32].values
y = dataset.iloc[:, 1].values

Comprobamos las dimensiones de nuestra matriz de entradas:

In [ ]:
X.shape

Ahora realizamos la codificación de datos de clasificación (la *M* y la *B* de las categorías de cancer) en valores binarios con los que podamos trabajar. Para ello, usamos la clase incluída en Scikit-Learn, `LabelEncoder`:

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

y = labelencoder.fit_transform(y)

Ya que sólo estamos trabajando con dos clases para la predicción, no es necesario realizar un *Hot Encoding* de las etiquetas.

Comprobamos las dimensiones del vector de etiquetas:

In [ ]:
y.shape

Ahora, realizamos el particionado de los datos de entrada en conjuntos de entrenamiento y test. Para ello usamos también una clase de Scikit-Learn, en este caso `train_test_split`, con una proporción 80/20 entre los datos de entrenamiento y los datos de test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

Una vez tenemos preparados los conjuntos de datos, hacemos también un escalado de las características para mejorar el comportamiento del entrenamiento:

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

## Entrenamiento de la red neuronal

Importamos los módulos de Keras necesarios para poder definir la arquitectura de nuestra red neuronal. Lo primero será importar el modelo secuencial que nos permite definir la red neuronal, y `Dense`, que nos permitirá configurar capas densas de nodos dentro de la red:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

Procedemos a inicializar un clasificador instanciando el modelo secuencial:

In [ ]:
classifier = Sequential()

Y ahora vamos añadiendo capas:
 - Entrada
 - Capas ocultas
 - Capa de salida

Inicializamos los valores de cada capa con una [distribución aleatoria uniforme](https://keras.io/api/layers/initializers/#randomuniform-class):


In [ ]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 30))

# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

Compilamos la red neuronal utilizando adam como optimizador y entropía cruzada binaria como función de pérdida:

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Y por último procedemos al entrenamiento de la red:

In [ ]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 80)

Realizamos la predicción sobre los valores de pruebas para poder componer la matriz de confusión:

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)   

Podemos ver que tenemos 70 True Negatives, 1 False Positive, 1 False Negative, y 42 True Positives:

In [ ]:
cm